# Chicago Crime Dataset Analysis

Dataset downloaded from: https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2

In [19]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from dython.nominal import associations

/home/mikexydas/pythonEnvs/thesisEnv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/mikexydas/pythonEnvs/thesisEnv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/mikexydas/pythonEnvs/thesisEnv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/mikexydas/pythonEnvs/thesisEnv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


Reading and printing the head of the file.
In order to save time I do not read all the columns. Some are redundant (eg FBI code, IUCR) or not actually usable such as having both ID and Case Number.

In [40]:
columns_to_read = ["ID", "Date", "Block", "IUCR", "Primary Type", "Description", "Location Description", "Arrest",
                  "Domestic", "Beat", "District", "Ward", "Community Area", "Year", "Latitude", "Longitude"]

rows_to_read = 2000 # In total 7000000 lines

print(">>> Started reading...")
df = pd.read_csv(filepath_or_buffer="datasets/Crimes_-_2001_to_present.csv", sep=",",
                    usecols=columns_to_read, parse_dates=["Date"], nrows = rows_to_read)

print(">>> Reading completed\n")
print(df.describe()) # Statistic information
print("")
print(df.head()) # Print first rows

>>> Started reading...
>>> Reading completed

                 ID         Beat     District         Ward  Community Area  \
count  2.000000e+03  2000.000000  2000.000000  1969.000000     1968.000000   
mean   1.086833e+07  1152.331000    11.258500    23.879126       37.344004   
std    1.422698e+06   720.121912     7.187733    14.613573       21.913822   
min    1.377897e+06   111.000000     1.000000     1.000000        1.000000   
25%    1.093966e+07   524.000000     5.000000     9.000000       23.000000   
50%    1.122858e+07  1034.000000    10.000000    24.000000       32.000000   
75%    1.125050e+07  1731.000000    17.000000    37.000000       56.000000   
max    1.164996e+07  2535.000000    25.000000    50.000000       77.000000   

              Year  Latitude  Longitude  
count  2000.000000       0.0        0.0  
mean   2014.877500       NaN        NaN  
std       3.495082       NaN        NaN  
min    2001.000000       NaN        NaN  
25%    2014.000000       NaN        NaN  

## Dealing with missing values

In [35]:
print(df.isnull().sum())

ID                          0
Date                        0
Block                       0
IUCR                        0
Primary Type                0
Description                 0
Location Description     2412
Arrest                      0
Domestic                    0
Beat                        0
District                    0
Ward                     9263
Community Area           9208
Year                        0
Latitude                66603
Longitude               66603
dtype: int64


In [42]:
# A naive approach of filling all the empty string with "" and all the other columns with 0

df["Location Description"] = df["Location Description"].fillna("")
df.fillna(0, inplace=True)

## Correlation Analysis

The dataset has some unwanted corellations. Some of them were easy to spot (IUCR and FBI code).
Now we must deal with more "hidden" realtionships.

The main difficulty is the variety of features (numerical and categorical).
After deciding what type is each column I used this package: https://github.com/shakedzy/dython/blob/master/

Column types:
* ID: Nominal (should be ignored)
* Date: Interval
* Block: Nominal
* IUCR: Nominal
* Primary Type: Nominal
* Description: Nominal
* Arrest: Nominal
* Domestic: Nominal
* Beat: Nominal
* District: Nominal
* Ward: Nominal
* Community Area: Nominal
* Year: Interval
* Latitude: Ratio
* Longitude: Ratio

In [43]:
from dython.nominal import theils_u
from dython.nominal import correlation_ratio

# Cast the True, False to 0, 1
df.Arrest = df.Arrest.astype(int)
df.Domestic = df.Domestic.astype(int)

# Convert the integer, float nominal variables to strings
nominal_cols = ["Arrest", "Domestic", "Beat", "District", "Ward", "Community Area"]
df = df.apply(lambda x: str(x) if x.name in nominal_cols else x)

# Convert the Date to timestamp (in seconds)
df.Date = df.Date.values.astype(np.int64) // 10 ** 9

print(df.dtypes)

# def choose_corr(col1_name, col2_name):
#     type1 = df

correlations = np.zeros((len(df.columns), len(df.columns)))



# for col_1_index in range(len(df.columns)):
#     for col_2_index in range(len(df.columns)):
        
        

ID                        int64
Date                      int64
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                   object
Domestic                 object
Beat                     object
District                 object
Ward                     object
Community Area           object
Year                      int64
Latitude                float64
Longitude               float64
dtype: object
